# Problem Statement

A small engineering consulting firm has 3 senior designers available to work on the firm's 4 current projects over the next 2 weeks. Each designer has 80 hours to split among the projects, and the following table shows the manager's scoring $(0=$ nil to $100=$ perfect $)$ of the capability of each designer to contribute to each project, along with his estimate of the hours that each project will require.


|      Designer     | Project 1    | Project 2    | Project 3    | Project 4    |
|----------|--------------|--------------|--------------|--------------|
| 1        | 90           | 80           | 10           | 50           |
| 2        | 60           | 70           | 50           | 65           |
| 3        | 70           | 40           | 80           | 85           |



|     **Required:**      | Project 1    | Project 2    | Project 3    | Project 4    |
|-----------|--------------|--------------|--------------|--------------|
| **Hours** | 70           | 50           | 85           | 35           |


#### Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import shutil
import sys
import os.path
from pyomo.environ import *

import pyomo.environ as pe
import pyomo.opt as po

#### Defining Data

In [2]:
Engg = {'E1','E2','E3'}    # Check the type by: type(Engg)

In [3]:
Proj = {'P1','P2','P3','P4'}

In [4]:
score = {
    ('E1','P1'):90,
    ('E1','P2'):80,
    ('E1','P3'):10,
    ('E1','P4'):50,
    ('E2','P1'):60,
    ('E2','P2'):70,
    ('E2','P3'):50,
    ('E2','P4'):65,
    ('E3','P1'):70,
    ('E3','P2'):40,
    ('E3','P3'):80,
    ('E3','P4'):85,
}   # Dictionary with tuples as keys (based on our defined sets)

In [5]:
hours_needed = {
    ('P1'):70,
    ('P2'):50,
    ('P3'):85,
    ('P4'):35,
}

In [6]:
max_hours = 80

## Model

Let the design engineers be set $E$ with $E_{i} \; :i \in [1,2,3]$ and the projects be $P$ with  $P_{j} \; :j \in [1,2,3,4]$. We can model the problem as allocation of the number of hours $H_{ij}$ with each design engineer $E_{i}$ $\forall i $ that are being put onto the projects $P_{j}$ $\forall j $, given the $i^{th}$ engineer $E$ works on $j^{th}$ project with given score $e_{ij}$.

Let the maximum hours available with each engineer be $H_{max}$ and the required number of hours for each project $P_j$ be $R_{j} \; \forall j $. Thus, then the mathematical formulation can be made as:

$$
\text{Maximize } \sum_{i \in E} \sum_{j \in P} H_{ij} e_{ij}
$$

Subject to:

$$
\sum_{j \in P} H_{ij} \leq H_{max} \;\; \forall i \in E
$$

$$
\sum_{i \in E} H_{ij} \geq R_j \;\; \forall j \in P
$$

$$
H_{ij} \geq 0 \;\; \forall i,j
$$


## Implement

In [7]:
m = pe.ConcreteModel()

#### Initializing Sets

In [8]:
m.Engg = pe.Set(initialize=Engg)
m.Proj = pe.Set(initialize=Proj)     # Ignore if any warnings appear

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


#### Initializing Parameters

In [9]:
m.score = pe.Param(m.Engg, m.Proj, initialize=score)
m.hours_needed = pe.Param(m.Proj, initialize=hours_needed)
m.max_hours = pe.Param(initialize=max_hours)

#### Initializing [Variables](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Sets.html) 
> Note: Refer the documnetation to also note other Predefined Virtual Sets

In [10]:
m.H = pe.Var(m.Engg, m.Proj, domain=pe.NonNegativeReals) 


#### Defining Objective

In [11]:
obj_expr = sum(m.H[i,j]*m.score[i,j] 
               for i in m.Engg for j in m.Proj)
m.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

#### Defining [Constraints](https://pyomo.readthedocs.io/en/stable/pyomo_modeling_components/Constraints.html)
> Note: Refer the documentation to see other ways to write the constraints, such as using a Constraint List

In [12]:
def maxhour_rule(m,i):
    return sum(m.H[i,j] for j in m.Proj) <= m.max_hours

m.maximum_hours =  pe.Constraint(m.Engg, rule=maxhour_rule)

In [13]:
def projhour_rule(m,j):  
    return sum(m.H[i,j] for i in m.Engg) >= m.hours_needed[j]

m.proj_hours = pe.Constraint(m.Proj, rule=projhour_rule)

## Solve and Postprocess

In [15]:
solver = po.SolverFactory('gurobi')
results = solver.solve(m, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-08
Read LP format model from file /var/folders/2v/vg66th916yb1cgzdj_0k857c0000gn/T/tmpcf_vfbl0.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 12 columns, 24 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x55465dd5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+32   1.200000e+31   7.500000e+02      0s
       7    1.8825000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.00 seconds (0.00 work units

In [16]:
print("Optimal Assignment Objective: ",pe.value(m.obj))

Optimal Assignment Objective:  18825.0


In [17]:
for i in m.Engg:
    for j in m.Proj:
        print("Engg",i," working on Project",j,"= ",pe.value(m.H[i,j])," Hours")


Engg E2  working on Project P1 =  0.0  Hours
Engg E2  working on Project P4 =  35.0  Hours
Engg E2  working on Project P2 =  40.0  Hours
Engg E2  working on Project P3 =  5.0  Hours
Engg E1  working on Project P1 =  70.0  Hours
Engg E1  working on Project P4 =  0.0  Hours
Engg E1  working on Project P2 =  10.0  Hours
Engg E1  working on Project P3 =  0.0  Hours
Engg E3  working on Project P1 =  0.0  Hours
Engg E3  working on Project P4 =  0.0  Hours
Engg E3  working on Project P2 =  0.0  Hours
Engg E3  working on Project P3 =  80.0  Hours


In [17]:
for i in m.Engg:
    h = pe.value(sum(m.H[i,j] for j in m.Proj))
    print("Hours Worked by Engineer",i,"= ",h," Hours")

Hours Worked by Engineer E3 =  80.0  Hours
Hours Worked by Engineer E2 =  80.0  Hours
Hours Worked by Engineer E1 =  80.0  Hours


In [23]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Sets for designers and projects
designers = [1, 2, 3]
projects = [1, 2, 3, 4]

# Capability scores (designer x project)
scores = {
    (1, 1): 90, (1, 2): 80, (1, 3): 10, (1, 4): 50,
    (2, 1): 60, (2, 2): 70, (2, 3): 50, (2, 4): 65,
    (3, 1): 70, (3, 2): 40, (3, 3): 80, (3, 4): 85
}

# Required hours for each project
required_hours = {1: 70, 2: 50, 3: 85, 4: 35}

# Maximum hours available per designer
max_hours_per_designer = 80

# Decision variables: Hours each designer works on each project
model.x = Var(designers, projects, bounds=(0, None), within=NonNegativeReals)

# Objective: Maximize the total capability score
model.obj = Objective(expr=sum(model.x[i, j] * scores[i, j] for i in designers for j in projects), sense=maximize)

# Constraint: Each project must receive exactly the required hours
model.project_constraints = ConstraintList()
for j in projects:
    model.project_constraints.add(sum(model.x[i, j] for i in designers) == required_hours[j])

# Constraint: Each designer can work at most 80 hours in total
model.designer_constraints = ConstraintList()
for i in designers:
    model.designer_constraints.add(sum(model.x[i, j] for j in projects) <= max_hours_per_designer)

# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model, tee=True)

# Display results
print("\nOptimal assignment of hours:")
for i in designers:
    for j in projects:
        print(f"Designer {i} -> Project {j}: {model.x[i, j].value} hours")
# Calculate and print total hours worked by each designer
print("\nTotal hours worked by each designer:")
for i in designers:
    total_hours = sum(model.x[i, j].value for j in projects)
    print(f"Designer {i}: {total_hours:.2f} hours")


print(f"\nTotal capability score: {model.obj()}")



Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-08
Read LP format model from file /var/folders/2v/vg66th916yb1cgzdj_0k857c0000gn/T/tmpr430592r.pyomo.lp
Reading time = 0.00 seconds
x1: 7 rows, 12 columns, 24 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x877c9312
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5000000e+32   2.400000e+31   7.500000e+02      0s
       7    1.8825000e+04   0.000000e+00   0.000000e+00      0s

Solved in 7 iterations and 0.01 seconds (0.00 work units